In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from src.data_processing.processors.TrainingProcessor import TrainingProcessor
from src.utilities.pandas_helpers import filter_columns_by_search

In [17]:
%load_ext autoreload 
%autoreload 2 

In [16]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## load preprocessed data

In [12]:
data = pd.read_parquet("/Users/mds8301/iterm_data_storage/full_dataset.parquet.gzip")

,time,sex_M,day,trial,trial_count,learning_phase,event_cue,event_escape,event_avoid,action_escape,...,mouse_id_4,mouse_id_5,mouse_id_6,mouse_id_7,mouse_id_8,mouse_id_9,mouse_id_10,mouse_id_11,mouse_id_12,mouse_id_13
0,-25.000000,0,5,0,115,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,-24.901531,0,5,0,115,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,-24.803064,0,5,0,115,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,-24.704596,0,5,0,115,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,-24.606127,0,5,0,115,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858795,19.606127,0,4,1,87,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5858796,19.704596,0,4,1,87,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5858797,19.803064,0,4,1,87,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5858798,19.901531,0,4,1,87,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data.columns

Index(['time', 'sex_M', 'day', 'trial', 'trial_count', 'learning_phase',
       'event_cue', 'event_escape', 'event_avoid', 'action_escape',
       'action_avoid', 'latency', 'event_shock', 'sensor_D1', 'sensor_D2',
       'sensor_DA', 'signal', 'mouse_id_0', 'mouse_id_1', 'mouse_id_2',
       'mouse_id_3', 'mouse_id_4', 'mouse_id_5', 'mouse_id_6', 'mouse_id_7',
       'mouse_id_8', 'mouse_id_9', 'mouse_id_10', 'mouse_id_11', 'mouse_id_12',
       'mouse_id_13'],
      dtype='object')

## agg data down to max and min for each day

In [ ]:
def group_data(df):
    events = 

    return (
        df.melt(id_vars = [])
])
    )

In [10]:
train_processor = (TrainingProcessor(data)
                .query_sensor_and_sort_trials_by_subject(sensor='DA')
                .split_train_val_test_by_subject(target='signal', shuffle=False)
                )

,time,sex_M,day,trial,trial_count,learning_phase,event_cue,event_escape,event_avoid,action_escape,...,mouse_id_4,mouse_id_5,mouse_id_6,mouse_id_7,mouse_id_8,mouse_id_9,mouse_id_10,mouse_id_11,mouse_id_12,mouse_id_13
1259063,-25.000000,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1259064,-24.901531,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1259065,-24.803064,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1259066,-24.704596,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1259067,-24.606127,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670342,19.606127,0,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1670343,19.704596,0,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1670344,19.803064,0,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1670345,19.901531,0,1,1,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
